<a href="https://colab.research.google.com/github/FariusGitHub/DataScience/blob/master/SOUP3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import bs4
import requests
from tqdm import tqdm
import pandas as pd
import numpy as np
from string import punctuation
from nltk.stem import PorterStemmer

soup = bs4.BeautifulSoup(requests.get(\
'https://www.monster.ca/jobs/search/?q='
                'data-scientist'
                '__2C-data-engineer'
                '__2C-machine-learning-engineer'
                '__2C-data-analyst'
                '&where=Canada&intcid=skr_navigation_nhpso_searchMain&page=10'\
                                     ).text, 'lxml')

company=[];location=[];title=[];jobage=[]; jobpage=[]; summary1=[]
for i,j in enumerate(tqdm(soup.select('.company'))):
            company  += [j.text.strip('\n')]
            location += [soup.select('.location')[i+1].text.strip('\n\r')]
            title    += [soup.select('.title')[i+3].text.strip('\r\n')]   
            jobage   += ['30 days or more' if soup.find_all(class_=\
                        'meta flex-col')[i].text[1:3]=='30' else\
                        'posted today'    if soup.find_all(class_=\
                        'meta flex-col')[i].text[1:3].isalpha() else\
                                            soup.find_all(class_=\
                        'meta flex-col')[i].text[1:3] + ' days ago']
            jobpage  += [soup.select('.title')[i+3].a.get('href')]
            #summary1 AS PER ERIC REQUEST: only have company/location info
            summary1 += [soup.select('.summary')[i]\
                         .find(class_='company').text.strip('\n')+", "+
                         soup.select('.summary')[i]\
                         .find(class_='location').text.strip('\n\r')]

def make_clickable(val):
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

monster = pd.DataFrame(np.asarray(list(zip(company, location, title, jobage, \
          jobpage, summary1))), columns =['Company', 'Location', 'Job Title',\
          'When Posted', 'Website', 'summary1']).drop_duplicates()\
          .reset_index(drop=True)

desc=[]
for i in tqdm(monster['Website']):
    try:
        desc += [bs4.BeautifulSoup(requests.get(i).text,\
                'lxml').select('.row')[3].text]
    except:
        desc += ['error during downloading']
        
#ADDING A NEW FEATURE USING MACHINE LEARNING/NLTK/STEMMING/PUNCTUATION REMOVAL          
jargons=pd.read_csv("https://farius.s3.amazonaws.com/jargons_01.csv", header=None)
ps = PorterStemmer()

summary2={};keywords1=[]; keywords2=[]; keywords3=[]
for k,j in enumerate(tqdm(desc)):
    cleaned = ''.join([char if char not in punctuation else ' '\
              for char in j.lower() ])
    keywords1=set([word for word in [cleaned.split()[i] \
              for i in range(len(cleaned.split()))] \
              if word in jargons[0].values.tolist()])
    keywords2=set([word for word in [cleaned.split()[i] +' '+ \
              cleaned.split()[i + 1] for i in range(len(cleaned.split()) - 1)] \
              if word in jargons[0].values.tolist()])
    keywords3=set([word for word in [cleaned.split()[i] +' '+ \
              cleaned.split()[i + 1] +' '+ ps.stem(cleaned.split()[i + 2]) \
              for i in range(len(cleaned.split()) - 2)] \
              if word in jargons[0].values.tolist()])
    summary2[k]=list(keywords1)+list(keywords2)+list(keywords3)
    
monster['summary2']=summary2.values()
monster.style.format({'Website': make_clickable})

# monster['description']=desc # given jargon library improved summary2 is better
monster.style.format({'Website': make_clickable})


100%|██████████| 170/170 [00:16<00:00, 10.52it/s]


,Company,Location,Job Title,When Posted,Website,summary1,summary2
0,EU Health and Innovations Inc,"Thornhill, ON",Systems Consultant & Data Analyst,8 days ago,https://job-openings.monster.ca/systems-consultant-data-analyst-thornhill-on-ca-eu-health-innovations-inc/221737818,"EU Health and Innovations Inc, Thornhill, ON","['statistical', 'sql', 'python', 'engineering', 'computer science']"
1,Geomodeling,"Calgary, AB",DATA SCIENTIST NOC 2172,20 days ago,https://job-openings.monster.ca/data-scientist-noc-2172-calgary-ab-ca-geomodeling/219706051,"Geomodeling, Calgary, AB","['visualization', 'python']"
2,Morgan Stanley,"Montreal, QC",BI/Data Analyst 3155721,1 days ago,https://job-openings.monster.ca/bi-data-analyst-3155721-montreal-qc-ca-morgan-stanley/222012028,"Morgan Stanley, Montreal, QC","['sql', 'building', 'visualization', 'engineering', 'statistics', 'tableau', 'python', 'computer science', 'machine learning']"
3,Juventus Consulting,"Markham, ON",Data Analyst,15 days ago,https://job-openings.monster.ca/data-analyst-markham-on-ca-juventus-consulting/221580192,"Juventus Consulting, Markham, ON","['r', 'tableau', 'sql']"
4,Randstad,"Toronto, ON",Data Scientist/Analyst Level 2 (Fraud Detection),25 days ago,https://job-openings.monster.ca/data-scientist-analyst-level-2-fraud-detection-toronto-on-ca-randstad/221247019,"Randstad, Toronto, ON","['statistical', 'building', 'python', 'data science', 'machine learning']"
5,Magna International Inc.,"Concord, ON",Junior BI Developer/Analyst,4 days ago,https://job-openings.monster.ca/junior-bi-developer-analyst-concord-on-ca-magna-international-inc/221670993,"Magna International Inc., Concord, ON","['training', 'sql', 'computer science']"
6,Prodigy Bank,"Toronto, ON",Data Scientist,9 days ago,https://job-openings.monster.ca/data-scientist-toronto-on-ca-prodigy-bank/221751235,"Prodigy Bank, Toronto, ON","['automl', 'visualization', 'training', 'classification', 'mathematics', 'regression', 'optimization', 'monitoring', 'databricks', 'unsupervised', 'sql', 'building', 'engineering', 'tableau', 'simulation', 'statistical', 'r', 'scoring', 'mathematical', 'scala', 'statistics', 'cluster', 'supervised', 'python', 'big data', 'advanced analytics', 'random forest', 'computer science', 'time series', 'feature engineering', 'data science', 'predictive modeling', 'predictive attributes', 'machine learning', 'machine learning concept', 'machine learning algorithm']"
7,ADGA Group,"Toronto, ON",Cognos/ETL Programmer/Developer,8 days ago,https://job-openings.monster.ca/cognos-etl-programmer-developer-toronto-on-ca-adga-group/221798449,"ADGA Group, Toronto, ON",[]
8,Robertson & Company Ltd.,"TORONTO, ON",Enterprise Full Stack Engineer – Big Data (perm),2 days ago,https://job-openings.monster.ca/enterprise-full-stack-engineer-–-big-data-perm-toronto-on-ca-robertson-company-ltd/221295753,"Robertson & Company Ltd., TORONTO, ON","['scala', 'python', 'sql', 'big data', 'machine learning']"
9,Ivanhoé Cambridge Inc,"Montréal, QC","Developer, Business Intelligence",25 days ago,https://job-openings.monster.ca/developer-business-intelligence-montréal-qc-ca-ivanhoé-cambridge-inc/221230575,"Ivanhoé Cambridge Inc, Montréal, QC","['monitoring', 'r', 'sql', 'visualization', 'training', 'python', 'data science']"
